In [0]:
%pip install -q -U apify-client

In [0]:
from apify_client import ApifyClient
from pyspark.sql.functions import col, current_timestamp, timestamp_diff

In [0]:
# Initialize the ApifyClient with your API token
client = ApifyClient(dbutils.secrets.get("job_notification", "Apify_Token"))
 
# Prepare the Actor input
run_input = {
    "country": "in",
    "datePosted": "1",
    "limit": 50,
    "location": "Hyderabad",
    "title": "Data Engineer"
}
 
# Run the Actor and wait for it to finish
run = client.actor("TrtlecxAsNRbKl1na").call(run_input=run_input)
data = client.dataset(run["defaultDatasetId"]).iterate_items()

In [0]:
l = []
for i in data:
    d = {}
    d['url'] = i['url']
    d['title'] = i['title']
    d['dateOnIndeed'] = i['dateOnIndeed']
    d['company'] = i['employer']['name']
    d['skills'] = str([i['attributes'][j] for j in i['attributes']])
    d['description'] = i['description']
    l.append(d)

In [0]:
df = spark.createDataFrame(l)
df = df.withColumn("hrs_ago", timestamp_diff("hour",  col("dateOnIndeed").cast("timestamp"), current_timestamp()))
df.write.mode('overwrite').option("mergeSchema", "true").saveAsTable("raw_catalogue.jobs.indeedjobs")
display(df)

In [0]:
display(l[40])
# url

In [0]:
df = spark.createDataFrame(l)
df = df.withColumn("hrs_ago", timestamp_diff("hour",  col("createdDate").cast("timestamp"), current_timestamp()))
df.write.mode('overwrite').option("mergeSchema", "true").saveAsTable("raw_catalogue.jobs.naukarijobs")
display(df)